 # Import Necessary Libraries

In [1]:
!pip install focal-loss
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import (Input, Conv2D, Conv2DTranspose, MaxPooling2D, 
                                     Concatenate, BatchNormalization, Activation)
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from focal_loss import SparseCategoricalFocalLoss
from tensorflow.keras import layers
from skimage import io


# Define Functions for Image Preprocessing

In [2]:
def load_and_split_image(image_path, img_size=(192, 288)):
    full_image = io.imread(image_path)  
    if full_image.shape[1] < 1140:
        print(f"Error: Image width too small - {image_path}, Shape: {full_image.shape}")
        return None, None
    
    bscan = full_image[:, :570, :]
    mask = full_image[:, 570:, :]
    
    bscan = tf.image.resize(bscan, img_size, method=tf.image.ResizeMethod.BILINEAR).numpy()
    mask = tf.image.resize(mask, img_size, method=tf.image.ResizeMethod.NEAREST_NEIGHBOR).numpy().astype(np.uint8)
    
    return bscan, mask

COLOR_MAP = {
    0: (255, 0, 0), 1: (255, 192, 203), 2: (0, 0, 255),
    3: (0, 255, 0), 4: (255, 255, 0), 5: (0, 0, 0)
}

# Convert RGB mask to integer label map (one channel: each pixel holds its class index)
def rgb_to_label(mask_rgb, tolerance=5):
    H, W, _ = mask_rgb.shape
    label_mask = np.zeros((H, W), dtype=np.uint8)
    for class_idx, rgb in COLOR_MAP.items():
        matches = np.all(np.abs(mask_rgb - np.array(rgb)) <= tolerance, axis=-1)
        label_mask[matches] = class_idx
    return label_mask

# Load Dataset

In [3]:
def load_dataset(txt_path, base_dir, img_size=(192, 288)):
    with open(txt_path, "r") as f:
        filenames = [line.strip() for line in f.readlines()]
    images, masks = [], []
    for filename in filenames:
        eye_id = filename.split("_")[0]
        image_path = os.path.join(base_dir, eye_id, filename)
        bscan, mask_rgb = load_and_split_image(image_path, img_size)
        if bscan is None or mask_rgb is None:
            continue
        mask_label = rgb_to_label(mask_rgb)
        images.append(bscan)
        masks.append(mask_label)
    return np.array(images), np.array(masks)

# Load your training and validation datasets
X_train, Y_train = load_dataset("/kaggle/input/amd-sd/AMD-SD/training.txt", "/kaggle/input/amd-sd/AMD-SD/images")
X_val, Y_val = load_dataset("/kaggle/input/amd-sd/AMD-SD/validation.txt", "/kaggle/input/amd-sd/AMD-SD/images")

# Normalize images; Y_train and Y_val are already integer labels of shape (batch, 192, 288)
X_train = X_train.astype("float32") / 255.0
X_val = X_val.astype("float32") / 255.0

# Basic convolution block for U-Net

In [4]:

def conv_block(inputs, num_filters):
    x = Conv2D(num_filters, (3, 3), padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = Conv2D(num_filters, (3, 3), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    return x

# Encoder block: conv_block then MaxPooling
def encoder_block(inputs, num_filters):
    x = conv_block(inputs, num_filters)
    p = MaxPooling2D((2,2))(x)
    return x, p


# Define Graph Attention Mechanism

In [5]:
# Graph Attention Layer (GAT)
class GraphAttentionLayer(layers.Layer):
    def __init__(self, channels, **kwargs):
        super(GraphAttentionLayer, self).__init__(**kwargs)
        self.channels = channels
        self.conv1 = Conv2D(channels, (1, 1), activation="relu")
        self.conv2 = Conv2D(channels, (1, 1), activation="sigmoid")
    
    def call(self, inputs):
        x = self.conv1(inputs)
        attention = self.conv2(x)
        return x * attention

# Decoder block: upsample, concatenate skip connection, conv_block, then Graph Attention

In [6]:

def decoder_block(inputs, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2,2), strides=2, padding="same")(inputs)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    x = GraphAttentionLayer(num_filters)(x)
    return x

# Self-routing Capsule Layer with dynamic routing

In [7]:

class SelfRoutingCapsuleLayer(layers.Layer):
    def __init__(self, num_capsules, capsule_dim, num_routing=3, **kwargs):
        super(SelfRoutingCapsuleLayer, self).__init__(**kwargs)
        self.num_capsules = num_capsules
        self.capsule_dim = capsule_dim
        self.num_routing = num_routing
        self.conv = Conv2D(num_capsules * capsule_dim, (3, 3), padding="same", activation="relu")
    
    def call(self, inputs):
        # inputs: (batch, H, W, channels)
        x = self.conv(inputs)  # (batch, H, W, num_capsules * capsule_dim)
        batch_size = tf.shape(x)[0]
        H = tf.shape(x)[1]
        W = tf.shape(x)[2]
        # Reshape to (batch, H*W, num_capsules, capsule_dim)
        x = tf.reshape(x, (batch_size, H * W, self.num_capsules, self.capsule_dim))
        # Initialize routing logits to zeros
        b = tf.zeros((batch_size, H * W, self.num_capsules))
        for i in range(self.num_routing):
            c = tf.nn.softmax(b, axis=-1)  # coupling coefficients
            s = tf.reduce_sum(tf.expand_dims(c, -1) * x, axis=1)  # (batch, num_capsules, capsule_dim)
            v = tf.nn.l2_normalize(s, axis=-1)  # squash activation
            b += tf.reduce_sum(x * tf.expand_dims(v, 1), axis=-1)
        # Reshape v to (batch, 1, 1, num_capsules * capsule_dim) and then tile spatially
        v = tf.reshape(v, (batch_size, 1, 1, self.num_capsules * self.capsule_dim))
        v = tf.tile(v, [1, H, W, 1])
        return v

# ResUNet with Graph Attention and Self-Routing Capsules

In [8]:

def ResUNet_GAT_SelfRoutingCapsule(input_shape=(192, 288, 3), num_classes=6):
    inputs = Input(input_shape)
    # Encoder
    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)
    # Bottleneck
    b = conv_block(p4, 1024)
    # Decoder
    d1 = decoder_block(b, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)
    # Self-routing Capsule Layer
    x = SelfRoutingCapsuleLayer(num_capsules=10, capsule_dim=16, num_routing=3)(d4)
    outputs = Conv2D(num_classes, (1, 1), activation="softmax", padding="same")(x)
    model = Model(inputs, outputs, name="ResUNet_GAT_SelfRoutingCapsule")
    return model


# CUSTOM METRICS (IOU, Dice, Hausdorff)

In [9]:

def iou_metric(y_true, y_pred, smooth=1e-6):
    y_true = tf.cast(y_true, tf.int32)
    y_pred = tf.argmax(y_pred, axis=-1)
    y_pred = tf.cast(y_pred, tf.int32)
    y_true_onehot = tf.one_hot(y_true, depth=6)
    y_pred_onehot = tf.one_hot(y_pred, depth=6)
    intersection = tf.reduce_sum(tf.cast(y_true_onehot * y_pred_onehot, tf.float32))
    union = tf.reduce_sum(tf.cast(y_true_onehot + y_pred_onehot, tf.float32)) - intersection
    return (intersection + smooth) / (union + smooth)

def dice_metric(y_true, y_pred, smooth=1e-6):
    y_true = tf.cast(y_true, tf.int32)
    y_pred = tf.argmax(y_pred, axis=-1)
    y_pred = tf.cast(y_pred, tf.int32)
    y_true_onehot = tf.one_hot(y_true, depth=6)
    y_pred_onehot = tf.one_hot(y_pred, depth=6)
    intersection = tf.reduce_sum(tf.cast(y_true_onehot * y_pred_onehot, tf.float32))
    dice = (2. * intersection + smooth) / (tf.reduce_sum(tf.cast(y_true_onehot, tf.float32)) + tf.reduce_sum(tf.cast(y_pred_onehot, tf.float32)) + smooth)
    return dice

from scipy.spatial.distance import directed_hausdorff
def compute_hausdorff_distance(y_true, y_pred, num_classes=6):
    hd_list = []
    for cls in range(num_classes):
        true_mask = (y_true == cls)
        pred_mask = (y_pred == cls)
        true_points = np.argwhere(true_mask)
        pred_points = np.argwhere(pred_mask)
        if true_points.size == 0 and pred_points.size == 0:
            hd = 0.0
        elif true_points.size == 0 or pred_points.size == 0:
            hd = 0.0
        else:
            d1 = directed_hausdorff(true_points, pred_points)[0]
            d2 = directed_hausdorff(pred_points, true_points)[0]
            hd = max(d1, d2)
        hd_list.append(hd)
    return np.nanmax(hd_list)

def hausdorff_distance_metric_numpy(y_true, y_pred, num_classes=6):
    y_pred = np.argmax(y_pred, axis=-1).astype(np.int32)
    y_true = y_true.astype(np.int32)
    hd_batch = []
    for i in range(y_true.shape[0]):
        hd = compute_hausdorff_distance(y_true[i], y_pred[i], num_classes=num_classes)
        hd_batch.append(hd)
    return np.array(hd_batch).mean().astype(np.float32)

def hausdorff_metric(y_true, y_pred):
    hd = tf.numpy_function(hausdorff_distance_metric_numpy, [y_true, y_pred, 6], tf.float32)
    hd.set_shape([])  # ensure the output shape is known
    return hd

In [10]:
from tensorflow.keras.layers import (
    Conv2D, MaxPooling2D, Dropout, BatchNormalization, 
    Activation, Input, Dense, Flatten, Concatenate,
    UpSampling2D  # Include this if you're using it in your U-Net style architecture
)
# Define a simpler CNN student model
def create_cnn_student_model(input_shape=(192, 288, 3), num_classes=6):
    """
    Creates a traditional CNN model for segmentation using a U-Net-like architecture
    but with standard convolutional blocks (no GAT or capsules).
    """
    # Input
    inputs = Input(input_shape)
    
    # Encoder path - traditional CNN blocks
    # Block 1
    conv1 = Conv2D(32, (3, 3), padding='same')(inputs)
    conv1 = BatchNormalization()(conv1)
    conv1 = Activation('relu')(conv1)
    conv1 = Conv2D(32, (3, 3), padding='same')(conv1)
    conv1 = BatchNormalization()(conv1)
    conv1 = Activation('relu')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    
    # Block 2
    conv2 = Conv2D(64, (3, 3), padding='same')(pool1)
    conv2 = BatchNormalization()(conv2)
    conv2 = Activation('relu')(conv2)
    conv2 = Conv2D(64, (3, 3), padding='same')(conv2)
    conv2 = BatchNormalization()(conv2)
    conv2 = Activation('relu')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    
    # Block 3
    conv3 = Conv2D(128, (3, 3), padding='same')(pool2)
    conv3 = BatchNormalization()(conv3)
    conv3 = Activation('relu')(conv3)
    conv3 = Conv2D(128, (3, 3), padding='same')(conv3)
    conv3 = BatchNormalization()(conv3)
    conv3 = Activation('relu')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    
    # Middle (bottleneck)
    conv4 = Conv2D(256, (3, 3), padding='same')(pool3)
    conv4 = BatchNormalization()(conv4)
    conv4 = Activation('relu')(conv4)
    conv4 = Conv2D(256, (3, 3), padding='same')(conv4)
    conv4 = BatchNormalization()(conv4)
    conv4 = Activation('relu')(conv4)
    conv4 = Dropout(0.5)(conv4)  # Add dropout for regularization
    
    # Decoder path
    # Block 3
    up3 = UpSampling2D(size=(2, 2))(conv4)
    up3 = Conv2D(128, (2, 2), padding='same')(up3)
    up3 = BatchNormalization()(up3)
    up3 = Activation('relu')(up3)
    merge3 = Concatenate()([conv3, up3])
    deconv3 = Conv2D(128, (3, 3), padding='same')(merge3)
    deconv3 = BatchNormalization()(deconv3)
    deconv3 = Activation('relu')(deconv3)
    deconv3 = Conv2D(128, (3, 3), padding='same')(deconv3)
    deconv3 = BatchNormalization()(deconv3)
    deconv3 = Activation('relu')(deconv3)
    
    # Block 2
    up2 = UpSampling2D(size=(2, 2))(deconv3)
    up2 = Conv2D(64, (2, 2), padding='same')(up2)
    up2 = BatchNormalization()(up2)
    up2 = Activation('relu')(up2)
    merge2 = Concatenate()([conv2, up2])
    deconv2 = Conv2D(64, (3, 3), padding='same')(merge2)
    deconv2 = BatchNormalization()(deconv2)
    deconv2 = Activation('relu')(deconv2)
    deconv2 = Conv2D(64, (3, 3), padding='same')(deconv2)
    deconv2 = BatchNormalization()(deconv2)
    deconv2 = Activation('relu')(deconv2)
    
    # Block 1
    up1 = UpSampling2D(size=(2, 2))(deconv2)
    up1 = Conv2D(32, (2, 2), padding='same')(up1)
    up1 = BatchNormalization()(up1)
    up1 = Activation('relu')(up1)
    merge1 = Concatenate()([conv1, up1])
    deconv1 = Conv2D(32, (3, 3), padding='same')(merge1)
    deconv1 = BatchNormalization()(deconv1)
    deconv1 = Activation('relu')(deconv1)
    deconv1 = Conv2D(32, (3, 3), padding='same')(deconv1)
    deconv1 = BatchNormalization()(deconv1)
    deconv1 = Activation('relu')(deconv1)
    
    # Output
    output = Conv2D(num_classes, (1, 1), activation='softmax')(deconv1)
    
    return Model(inputs=inputs, outputs=output, name="CNN_Student_Model")

In [11]:
from tensorflow.keras.models import load_model
import tensorflow as tf

class WeightedSparseCategoricalFocalLoss(tf.keras.losses.Loss):
    def __init__(self, class_weights=[3.0, 3.0, 3.0, 4.0, 3.0, 1.0], gamma=2, **kwargs):
        super().__init__(**kwargs)
        self.class_weights = tf.constant(class_weights, dtype=tf.float32)
        self.gamma = gamma
        self.__name__ = 'weighted_sparse_categorical_focal_loss'
    
    def __call__(self, y_true, y_pred, sample_weight=None):
        y_true_int = tf.cast(tf.reshape(y_true, [-1]), tf.int32)
        y_pred = tf.reshape(y_pred, [-1, tf.shape(y_pred)[-1]])
        
        ce_loss = tf.keras.losses.sparse_categorical_crossentropy(y_true_int, y_pred, from_logits=False)
        
        y_pred_soft = tf.nn.softmax(y_pred)
        y_true_onehot = tf.one_hot(y_true_int, depth=tf.shape(y_pred)[-1])
        p_t = tf.reduce_sum(y_pred_soft * y_true_onehot, axis=-1)
        focal_factor = tf.pow(1 - p_t, self.gamma)
        
        weights = tf.gather(self.class_weights, y_true_int)
        loss = weights * focal_factor * ce_loss
        return tf.reduce_mean(loss)
    
    def get_config(self):
        config = super().get_config()
        config.update({
            "class_weights": self.class_weights.numpy().tolist(),
            "gamma": self.gamma
        })
        return config
    
    @classmethod
    def from_config(cls, config):
        # Extract class_weights and gamma if they exist, otherwise use defaults
        class_weights = config.pop("class_weights", [3.0, 3.0, 3.0, 4.0, 3.0, 1.0])
        gamma = config.pop("gamma", 2)
        return cls(class_weights=class_weights, gamma=gamma, **config)

In [12]:
class EnhancedDistiller(tf.keras.Model):
    def __init__(self, student, teacher, alpha=0.5, temperature=3, feature_distillation=False, **kwargs):
        super(EnhancedDistiller, self).__init__(**kwargs)
        self.student = student
        self.teacher = teacher
        self.alpha = alpha
        self.temperature = temperature
        self.feature_distillation = feature_distillation
        
        # Initialize loss functions
        self.class_weights = [3.0, 3.0, 3.0, 4.0, 3.0, 1.0]
        self.student_loss_fn = WeightedSparseCategoricalFocalLoss(self.class_weights, gamma=2)
        self.distillation_loss_fn = tf.keras.losses.KLDivergence()
        
        # For feature distillation (optional)
        if feature_distillation:
            self.feature_loss_fn = tf.keras.losses.MeanSquaredError()
    
    def compile(self, optimizer, metrics=None, **kwargs):
        super(EnhancedDistiller, self).compile(optimizer=optimizer, metrics=metrics, **kwargs)
    
    def call(self, inputs, training=False):
        return self.student(inputs, training=training)
    
    def train_step(self, data):
        x, y = data
        teacher_predictions = self.teacher(x, training=False)
        
        with tf.GradientTape() as tape:
            # Get student predictions
            student_predictions = self.student(x, training=True)
            
            # Calculate student loss (ground truth loss)
            student_loss = self.student_loss_fn(y, student_predictions)
            
            # Knowledge distillation loss
            teacher_soft = tf.nn.softmax(teacher_predictions / self.temperature)
            student_soft = tf.nn.softmax(student_predictions / self.temperature)
            distillation_loss = self.distillation_loss_fn(teacher_soft, student_soft)
            
            # Combined loss
            loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss
        
        # Apply gradients
        gradients = tape.gradient(loss, self.student.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.student.trainable_variables))
        
        # Update metrics
        self.compiled_metrics.update_state(y, student_predictions)
        results = {m.name: m.result() for m in self.metrics}
        results.update({
            "loss": loss,
            "student_loss": student_loss,
            "distillation_loss": distillation_loss
        })
        return results
    
    def test_step(self, data):
        x, y = data
        student_predictions = self.student(x, training=False)
        loss = self.student_loss_fn(y, student_predictions)
        self.compiled_metrics.update_state(y, student_predictions)
        results = {m.name: m.result() for m in self.metrics}
        results.update({"loss": loss})
        return results

# Compile and train the model

In [ ]:
class_weights = [3.0, 3.0, 3.0, 4.0, 3.0, 1.0]
loss_instance = WeightedSparseCategoricalFocalLoss(class_weights, gamma=2)

teacher_model = ResUNet_GAT_SelfRoutingCapsule()
teacher_model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss=loss_instance,
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy(), iou_metric, dice_metric]
)
teacher_model.fit(X_train, Y_train, epochs=30, batch_size=8, validation_data=(X_val, Y_val))
teacher_model.save("teacher_model.h5")  # Save the trained teacher model


Epoch 1/30


In [ ]:
# Example code to load the teacher model and create a student for distillation
def setup_distillation(teacher_model_path="teacher_model.h5"): 
    # Create the loss function with the same name as in the saved model
    class_weights = [3.0, 3.0, 3.0, 4.0, 3.0, 1.0]
    loss_instance = WeightedSparseCategoricalFocalLoss(class_weights, gamma=2)
    
    # Important: Create the custom objects dictionary with both the class AND function
    custom_objects = {
        "GraphAttentionLayer": GraphAttentionLayer,
        "SelfRoutingCapsuleLayer": SelfRoutingCapsuleLayer,
        "iou_metric": iou_metric, 
        "dice_metric": dice_metric,
        "weighted_sparse_categorical_focal_loss": loss_instance,
        "WeightedSparseCategoricalFocalLoss": WeightedSparseCategoricalFocalLoss
    }
    # Load the teacher model
    teacher_model = tf.keras.models.load_model(teacher_model_path, custom_objects=custom_objects)
    
    # Create a simpler CNN student model
    student_model = create_cnn_student_model()
    
    # Create the distiller
    distiller = EnhancedDistiller(
        student=student_model,
        teacher=teacher_model,
        alpha=0.6,  # Weight balance between student loss and distillation loss
        temperature=2  # Higher temperature produces softer probability distributions
    )
    
    # Compile the distiller
    distiller.compile(
        optimizer=Adam(learning_rate=1e-4),
        metrics=[
            tf.keras.metrics.SparseCategoricalAccuracy(),
            iou_metric,
            dice_metric
        ]
    )
    
    return distiller

In [ ]:
def train_distillation_model(distiller, X_train, Y_train, X_val, Y_val, epochs=15, batch_size=8):
    # Add learning rate scheduling for better convergence
    lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss', 
        factor=0.5,
        patience=3,
        min_lr=1e-6,
        verbose=1
    )
    
    # Early stopping to prevent overfitting
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=15,
        restore_best_weights=True,
        verbose=1
    )
    
    # Model checkpoint to save the best model - UPDATED FILE EXTENSION
    model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
        'best_student_model.keras', 
        monitor='val_loss',
        save_best_only=True,
        verbose=1
    )
    
    # Train the model with distillation
    history = distiller.fit(
        X_train, Y_train,
        validation_data=(X_val, Y_val),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=[lr_scheduler, early_stopping, model_checkpoint]
    )
    
    # Save the final student model - UPDATED FILE EXTENSION
    distiller.student.save('best_student_model.h5')  # Changed from .h5 to .keras
    
    return history, distiller.student

In [ ]:
distiller = setup_distillation()

# 2. Load or prepare your data
# For example:
# X_train, Y_train = load_training_data()
# X_val, Y_val = load_validation_data()

# 3. Call the training function with your data
history, trained_student_model = train_distillation_model(
    distiller,
    X_train, Y_train,
    X_val, Y_val,
    epochs=20,
    batch_size=8
)
distiller.student.save('best_student_model.keras') 
print("Training completed. Model saved as 'final_student_model.h5'")


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Convert training history dictionary to DataFrame
history_df = pd.DataFrame(history.history).round(4)

# Print the metrics table
print("Training Metrics per Epoch:")
print(history_df)

# Save the metrics table as a CSV file
history_df.to_csv("training_metrics.csv", index=False)
print("Metrics table saved to 'training_metrics.csv'.")

# Plot Training & Validation Loss
plt.figure(figsize=(8, 6))
plt.plot(history_df['loss'], label='Training Loss')
if 'val_loss' in history_df.columns:
    plt.plot(history_df['val_loss'], label='Validation Loss')
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training vs Validation Loss")
plt.legend()
plt.grid(True)
plt.savefig("loss_plot.png")
plt.show()
print("Loss plot saved to 'loss_plot.png'.")

# Plot Training & Validation Accuracy
plt.figure(figsize=(8, 6))
plt.plot(history_df['sparse_categorical_accuracy'], label='Training Accuracy')
if 'val_sparse_categorical_accuracy' in history_df.columns:
    plt.plot(history_df['val_sparse_categorical_accuracy'], label='Validation Accuracy')
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.title("Training vs Validation Accuracy")
plt.legend()
plt.grid(True)
plt.savefig("accuracy_plot.png")
plt.show()
print("Accuracy plot saved to 'accuracy_plot.png'.")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random

# Define the color map (mapping class indices to RGB tuples)
COLOR_MAP = {
    0: (255, 0, 0),      # e.g., class 0: red
    1: (255, 192, 203),  # class 1: pink
    2: (0, 0, 255),      # class 2: blue
    3: (0, 255, 0),      # class 3: green
    4: (255, 255, 0),    # class 4: yellow
    5: (0, 0, 0)         # class 5: black
}

#Function to convert an RGB mask to a label mask (integer labels)
def rgb_to_label(mask_rgb, tolerance=5):
    H, W, _ = mask_rgb.shape
    label_mask = np.zeros((H, W), dtype=np.uint8)
    for class_idx, rgb in COLOR_MAP.items():
        # Create a boolean mask where the difference between mask_rgb and rgb is within tolerance
        matches = np.all(np.abs(mask_rgb - np.array(rgb)) <= tolerance, axis=-1)
        label_mask[matches] = class_idx
    return label_mask

#Function to convert a label mask to an RGB mask using the COLOR_MAP
def label_to_rgb(label_mask):
    H, W = label_mask.shape
    rgb_mask = np.zeros((H, W, 3), dtype=np.uint8)
    for class_idx, color in COLOR_MAP.items():
        rgb_mask[label_mask == class_idx] = color
    return rgb_mask

# Manually set the index of the validation image to inspect
manual_idx = 6 # Change this value to select a different image

# Check index range
if manual_idx < 0 or manual_idx >= len(X_val):
    print("Index out of range. Please choose an index between 0 and", len(X_val)-1)
else:
    # Retrieve the OCT scan and the corresponding ground truth mask
    # Retrieve the OCT scan and the corresponding ground truth mask
    oct_scan = X_val[manual_idx]       # Shape: (192, 288, 3)
    true_mask = Y_val[manual_idx]      # Shape: (192, 288) or (192, 288, 3)
    
    # Check if true_mask is already a label mask
    if len(true_mask.shape) == 3:  # Only convert if it's an RGB image
        converted_labels = rgb_to_label(true_mask)
    else:
        converted_labels = true_mask  # Already in label format
    
    print("Unique labels in converted mask:", np.unique(converted_labels))

    # Convert labels to RGB for visualization
    true_mask_rgb = label_to_rgb(converted_labels)
    
    # Predict the segmentation mask using the trained model (distiller)
    #oct_scan = oct_scan.astype("float32") / 255.0  # Normalize to [0, 1]
    predicted = distiller.predict(np.expand_dims(oct_scan, axis=0))
    
    print("Prediction shape:", predicted.shape)
    print("Prediction min:", np.min(predicted), "max:", np.max(predicted))
    print("Prediction mean:", np.mean(predicted))

    predicted_mask = np.argmax(predicted[0], axis=-1)  # Shape: (192, 288)
    
    # Debug Step: Check if the model is predicting valid classes
    print("Unique values in predicted mask:", np.unique(predicted_mask))
    
    # Convert predicted labels to RGB for visualization
    predicted_mask_rgb = label_to_rgb(predicted_mask)

    # Plot the original OCT scan, ground truth colored mask, and predicted colored mask
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    
    axes[0].imshow(oct_scan)
    axes[0].set_title("Original OCT Scan")
    axes[0].axis("off")
    
    axes[1].imshow(true_mask_rgb)
    axes[1].set_title("Ground Truth Segmentation (Colored)")
    axes[1].axis("off")
    
    axes[2].imshow(predicted_mask_rgb)
    axes[2].set_title("Predicted Segmentation (Colored)")
    axes[2].axis("off")
    
    plt.tight_layout()
    plt.show()



In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

def get_grad_cam(model, img_array, layer_name=None):
    # Find the last convolutional layer in the encoder
    if layer_name is None:
        for layer in model.layers[::-1]:
            if 'conv' in layer.name.lower():
                layer_name = layer.name
                break

    grad_model = tf.keras.models.Model(
        inputs=model.inputs,
        outputs=[model.get_layer(layer_name).output, model.output]
    )

    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        loss = predictions[:, :, :, tf.argmax(predictions[0], axis=-1)[0]]  # Target class

    grads = tape.gradient(loss, conv_outputs)[0]
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1))

    conv_outputs = conv_outputs[0]
    heatmap = tf.reduce_sum(tf.multiply(pooled_grads, conv_outputs), axis=-1)
    heatmap = np.maximum(heatmap, 0)  # ReLU
    heatmap /= np.max(heatmap) if np.max(heatmap) > 0 else 1
    return heatmap.numpy()

# Example usage
sample_idx = 0
img = X_val[sample_idx][np.newaxis, ...]
heatmap = get_grad_cam(distiller.student, img)

plt.imshow(X_val[sample_idx])
plt.imshow(heatmap, alpha=0.5, cmap='jet')
plt.title("Grad-CAM Heatmap")
plt.show()